## LIMPIEZA DE DATOS ARCHIVOS CSV:


(Pendiente explicación de lo que contienen los archivos)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../src')
from suport import unif_col
from suport import lower_tildes
from suport import clean_year
from suport import trimestre
from suport import eliminar_alfa  
from unidecode import unidecode
pd.set_option('display.max_columns', None) 

### 1. LIMPIEZA ARCHIVO DENUNCIAS POR VIOLENCIA DE GÉNERO:


In [2]:
denu = pd.read_csv('../data/denuncias.csv')
denu.head()

,Año,Trimestre,Origen de la denuncia,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Gipuzkoa,Bizkaia,La Rioja,Ceuta,Melilla
0,Año 2009,Primero,Presentada directamente por victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,Año 2009,Primero,Presentada directamente por familiares,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,Primero,Atestados policiales - con denuncia victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,Año 2009,Primero,Atestados policiales - con denuncia familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,Año 2009,Primero,Atestados policiales - por intervención direct...,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [3]:
denu.columns = unif_col(denu.columns)
denu['trimestre'] = lower_tildes(denu['trimestre'])
denu.isna().value_counts()

ano    trimestre  origen de la denuncia  almeria  cadiz  cordoba  granada  huelva  jaen   malaga  sevilla  huesca  teruel  zaragoza  asturias  illes balears  las palmas  santa cruz de tenerife  cantabria  avila  burgos  leon   palencia  salamanca  segovia  soria  valladolid  zamora  albacete  ciudad real  cuenca  guadalajara  toledo  barcelona  girona  lleida  tarragona  alicante/alacant  castellon/castello  valencia/valencia  badajoz  caceres  a coruna  lugo   ourense  pontevedra  madrid  murcia  navarra  araba/alava  gipuzkoa  bizkaia  la rioja  ceuta  melilla
False  False      False                  False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              

In [4]:
denu.rename(columns={'ano': 'año'}, inplace=True)
denu.año = clean_year(denu.año)

In [5]:
denu.rename(columns={'origen de la denuncia': 'origen_denuncia'}, inplace=True)
denu['origen_denuncia'] = lower_tildes(denu['origen_denuncia'])

In [6]:
denu['origen_denuncia'].unique()

array(['presentada directamente por victima',
       'presentada directamente por familiares',
       'atestados policiales - con denuncia victima',
       'atestados policiales - con denuncia familiar',
       'atestados policiales - por intervencion directa policial',
       'parte de lesiones', 'servicios asistencia-terceros  en general'],
      dtype=object)

In [7]:
origen = {'presentada directamente por victima': 'victima',
          'presentada directamente por familiares': 'familiar',
          'atestados policiales - con denuncia victima': 'policia_victima',
          'atestados policiales - con denuncia familiar': 'policia_familiar',
          'atestados policiales - por intervencion directa policial': 'intervencion_policial_directa',
          'parte de lesiones': 'parte_lesiones',
          'servicios asistencia-terceros  en general': 'asistencia_terceros'}

denu['origen_denuncia'] = denu['origen_denuncia'].map(origen)

In [8]:
denu.head()

,año,trimestre,origen_denuncia,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,gipuzkoa,bizkaia,la rioja,ceuta,melilla
0,2009,primero,victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,2009,primero,familiar,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2009,primero,policia_victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,2009,primero,policia_familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,2009,primero,intervencion_policial_directa,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [9]:
# Guardamos el dataframe en un archivo csv:

denu.to_csv('../data_clean/portal_estadistico_vio_gen/denuncias.csv', index=False)

### 2. LIMPIEZA ARCHIVO LLAMADAS 016:

In [10]:
llam = pd.read_csv('../data/llamadas.csv')
llam.head()

,Año,Trimestre,Llamante,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla,No consta
0,Año 2007,Tercero,Usuaria,3.0,2.0,7.0,6.0,4.0,NaN,8.0,16.0,1.0,NaN,4.0,5.0,6.0,20.0,6.0,8.0,3.0,NaN,2.0,2.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,NaN,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,NaN,1.0,1.0,16.0
1,Año 2007,Tercero,Familiares/Personas allegadas,NaN,NaN,3.0,1.0,2.0,NaN,5.0,9.0,NaN,NaN,NaN,2.0,1.0,3.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,1.0,5.0,NaN,4.0,2.0,1.0,1.0,NaN,NaN,1.0,30.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,3.0
2,Año 2007,Tercero,Otros,NaN,NaN,NaN,1.0,2.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,4.0,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,20.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0
3,Año 2007,Cuarto,Usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,Año 2007,Cuarto,Familiares/Personas allegadas,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [11]:
llam.columns = unif_col(llam.columns)
llam['trimestre'] = lower_tildes(llam['trimestre'])
llam.llamante = lower_tildes(llam.llamante)
llam.rename(columns={'ano': 'año'}, inplace=True)
llam.año = clean_year(llam.año)

In [12]:
llam.isna().value_counts()

año    trimestre  llamante  almeria  cadiz  cordoba  granada  huelva  jaen   malaga  sevilla  huesca  teruel  zaragoza  asturias  illes balears  las palmas  santa cruz de tenerife  cantabria  avila  burgos  leon   palencia  salamanca  segovia  soria  valladolid  zamora  albacete  ciudad real  cuenca  guadalajara  toledo  barcelona  girona  lleida  tarragona  alicante/alacant  castellon/castello  valencia/valencia  badajoz  caceres  a coruna  lugo   ourense  pontevedra  madrid  murcia  navarra  araba/alava  bizkaia  gipuzkoa  la rioja  ceuta  melilla  no consta
False  False      False     False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              False    False 

In [13]:
llam.llamante.unique()

array(['usuaria', 'familiares/personas allegadas', 'otros', 'desconocido',
       'no consta'], dtype=object)

In [14]:
llam.columns

Index(['año', 'trimestre', 'llamante', 'almeria', 'cadiz', 'cordoba',
       'granada', 'huelva', 'jaen', 'malaga', 'sevilla', 'huesca', 'teruel',
       'zaragoza', 'asturias', 'illes balears', 'las palmas',
       'santa cruz de tenerife', 'cantabria', 'avila', 'burgos', 'leon',
       'palencia', 'salamanca', 'segovia', 'soria', 'valladolid', 'zamora',
       'albacete', 'ciudad real', 'cuenca', 'guadalajara', 'toledo',
       'barcelona', 'girona', 'lleida', 'tarragona', 'alicante/alacant',
       'castellon/castello', 'valencia/valencia', 'badajoz', 'caceres',
       'a coruna', 'lugo', 'ourense', 'pontevedra', 'madrid', 'murcia',
       'navarra', 'araba/alava', 'bizkaia', 'gipuzkoa', 'la rioja', 'ceuta',
       'melilla', 'no consta'],
      dtype='object')

In [15]:
quien_llama = {'usuaria': 'usuaria',
               'familiares/personas allegadas': 'familiar',
               'otros': 'desconocido',
               'desconocido': 'desconocido',
               'no Consta': 'desconocido'}

llam['llamante'] = llam['llamante'].map(quien_llama)

In [16]:
llam.fillna(0, inplace = True)

llam.head()

,año,trimestre,llamante,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla,no consta
0,2007,tercero,usuaria,3.0,2.0,7.0,6.0,4.0,0.0,8.0,16.0,1.0,0.0,4.0,5.0,6.0,20.0,6.0,8.0,3.0,0.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,0.0,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,0.0,1.0,1.0,16.0
1,2007,tercero,familiar,0.0,0.0,3.0,1.0,2.0,0.0,5.0,9.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,5.0,0.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,30.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,3.0
2,2007,tercero,desconocido,0.0,0.0,0.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0
3,2007,cuarto,usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,2007,cuarto,familiar,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [17]:
llam.to_csv('../data_clean/portal_estadistico_vio_gen/llamadas_016.csv', index=False)

### 3. LIMPIEZA ARCHIVO ORDEN PROTECCIÓN

In [18]:
ord = pd.read_csv('../data/orden_proteccion.csv')
ord.head()

,Año,Trimestre,Provincia,Número de órdenes de protección
0,Año 2009,Primero,Almería,82.0
1,Año 2009,Primero,Cádiz,312.0
2,Año 2009,Primero,Córdoba,125.0
3,Año 2009,Primero,Granada,163.0
4,Año 2009,Primero,Huelva,138.0


In [19]:
ord.columns = unif_col(ord.columns)
ord.rename(columns={'ano': 'año'}, inplace=True)
ord.rename(columns={'numero de ordenes de proteccion': 'numero_ordenes_proteccion'}, inplace=True)
ord.año = clean_year(ord.año)

In [20]:
ord.trimestre = lower_tildes(ord.trimestre)
ord.provincia = lower_tildes(ord.provincia)
ord.isna().value_counts()

año    trimestre  provincia  numero_ordenes_proteccion
False  False      False      False                        3016
Name: count, dtype: int64

In [21]:
ord.head()

,año,trimestre,provincia,numero_ordenes_proteccion
0,2009,primero,almeria,82.0
1,2009,primero,cadiz,312.0
2,2009,primero,cordoba,125.0
3,2009,primero,granada,163.0
4,2009,primero,huelva,138.0


In [22]:
# Guardamos el dataframe en un nuevo archivo csv

ord.to_csv('../data_clean/portal_estadistico_vio_gen/ordenes_prot.csv', index=False)

### 4. LIMPIEZA ARCHIVO DISPOSITIVOS DE SEGUIMIENTO

In [23]:
disp = pd.read_csv('../data/dispositivos.csv')
disp.head()

,Año,Mes,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla
0,Año 2009,Agosto,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Año 2009,Septiembre,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,Octubre,1.0,1.0,1.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,0.0,2.0,8.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,Año 2009,Noviembre,1.0,1.0,1.0,0.0,1.0,5.0,2.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,5.0,2.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0
4,Año 2009,Diciembre,1.0,1.0,2.0,2.0,1.0,8.0,3.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,5.0,1.0,8.0,3.0,0.0,2.0,1.0,0.0,1.0,88.0,0.0,1.0,0.0,7.0,1.0,0.0,0.0,0.0


In [24]:
disp.isna().value_counts()

Año    Mes    Almería  Cádiz  Córdoba  Granada  Huelva  Jaén   Málaga  Sevilla  Huesca  Teruel  Zaragoza  Asturias  Illes Balears  Las Palmas  Santa Cruz de Tenerife  Cantabria  Ávila  Burgos  León   Palencia  Salamanca  Segovia  Soria  Valladolid  Zamora  Albacete  Ciudad Real  Cuenca  Guadalajara  Toledo  Barcelona  Girona  Lleida  Tarragona  Alicante/Alacant  Castellón/Castelló  Valencia/València  Badajoz  Cáceres  A Coruña  Lugo   Ourense  Pontevedra  Madrid  Murcia  Navarra  Araba/Álava  Bizkaia  Gipuzkoa  La Rioja  Ceuta  Melilla
False  False  False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              False    False    False     False  False    False     

In [25]:
disp.columns = unif_col(disp.columns)
disp.mes = lower_tildes(disp.mes)
disp.rename(columns={'ano': 'año'}, inplace=True)
disp.año = clean_year(disp.año)

In [26]:
# Añadimos columna de trimestre:
disp['trimestre'] = disp['mes'].apply(trimestre)

# Vamos a reordenar las columnas:
columnas = disp.columns.tolist()

# Extraemos la última columna de trimestre:
ultima_columna = columnas[-1]

# La movemos a la tercera posición para que tenga consistencia:
columnas.insert(2, columnas.pop(-1))

# Reordenamos:
disp = disp[columnas]

In [27]:
disp.head()

,año,mes,trimestre,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla
0,2009,agosto,tercero,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009,septiembre,tercero,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2009,octubre,cuarto,1.0,1.0,1.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,0.0,2.0,8.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,2009,noviembre,cuarto,1.0,1.0,1.0,0.0,1.0,5.0,2.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,5.0,2.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0
4,2009,diciembre,cuarto,1.0,1.0,2.0,2.0,1.0,8.0,3.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,5.0,1.0,8.0,3.0,0.0,2.0,1.0,0.0,1.0,88.0,0.0,1.0,0.0,7.0,1.0,0.0,0.0,0.0


In [28]:
# Guardamos el nuevo dataframe en un archivo csv

disp.to_csv('../data_clean/portal_estadistico_vio_gen/dispositivos.csv', index=False)

### 5. LIMPIEZA ARCHIVO CSV MENORES VÍCTIMAS MORTALES DE VIOLENCIA DE GÉNERO:

In [29]:
men = pd.read_csv('../data/menores.csv')
men.head()

,Año,Trimestre,Tramo de edad de la víctima,Padre biológico,Suicidio,Almería,Cádiz,Jaén,Málaga,Teruel,Asturias,Illes Balears,Santa Cruz de Tenerife,Cantabria,Valladolid,Ciudad Real,Barcelona,Alicante/Alacant,Castellón/Castelló,Valencia/València,A Coruña,Pontevedra,Madrid,Murcia
0,Año 2013,Primero,3-4 años,Sí,No hubo tentativa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Año 2013,Segundo,<1 año,Sí,Suicidio consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Año 2013,Segundo,5-6 años,Sí,Suicidio consumado,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Año 2013,Segundo,11-12 años,Sí,Suicidio consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Año 2013,Tercero,5-6 años,Sí,No hubo tentativa,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
men.columns = unif_col(men.columns)
men.rename(columns={'padre biologico': 'padre_biologico'}, inplace=True)
men.rename(columns={'ano': 'año'}, inplace=True)
men.rename(columns={'tramo de edad de la victima': 'edad'}, inplace=True)

men['padre_biologico'] = lower_tildes(men['padre_biologico'])
men.trimestre = lower_tildes(men.trimestre)
men.suicidio = lower_tildes(men.suicidio)
men.edad = eliminar_alfa(men.edad)
men.año = clean_year(men.año)
men.fillna(0, inplace = True)

In [31]:
men.suicidio.unique()

array(['no hubo tentativa', 'suicidio consumado',
       'tentativa no consumada'], dtype=object)

In [32]:
suicidio = {'no hubo tentativa': 'no',
               'suicidio consumado': 'si',
               'tentativa no consumada': 'tentativa'}

men.suicidio = men.suicidio.map(suicidio)

In [33]:
men.head()

,año,trimestre,edad,padre_biologico,suicidio,almeria,cadiz,jaen,malaga,teruel,asturias,illes balears,santa cruz de tenerife,cantabria,valladolid,ciudad real,barcelona,alicante/alacant,castellon/castello,valencia/valencia,a coruna,pontevedra,madrid,murcia
0,2013,primero,3-4,si,no,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013,segundo,<1,si,si,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013,segundo,5-6,si,si,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013,segundo,11-12,si,si,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013,tercero,5-6,si,no,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
men.to_csv('../data_clean/portal_estadistico_vio_gen/menores.csv', index=False)

### 6. LIMPIEZA ARCHIVO CSV VÍCTIMAS MORTALES POR VIOLENCIA DE GÉNERO

In [35]:
vic = pd.read_csv('../data/victimas_mortales.csv')
vic.head()

,Año,Mes,Pareja - Expareja,Convivencia,Denuncia agresor,Suicidio,Tramo de edad Agresor,Tramo de edad Víctima,País de nacimiento Agresor,País de nacimiento Víctima,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla
0,Año 2003,Enero,Expareja,No,No consta denuncia,No hubo tentativa,No consta,31-40 años,España,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,31-40 años,31-40 años,Otro país,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,51-60 años,41-50 años,España,España,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,No consta,21-30 años,No consta,No consta,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,No consta,31-40 años,España,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Formateamos sin minusculas ni tildes y modificamos nulos:
vic.fillna(0, inplace = True)
vic = vic.apply(lower_tildes)
vic.columns = unif_col(vic.columns)
vic.head()

,ano,mes,pareja - expareja,convivencia,denuncia agresor,suicidio,tramo de edad agresor,tramo de edad victima,pais de nacimiento agresor,pais de nacimiento victima,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla
0,ano 2003,enero,expareja,no,no consta denuncia,no hubo tentativa,no consta,31-40 anos,espana,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,31-40 anos,31-40 anos,otro pais,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,51-60 anos,41-50 anos,espana,espana,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,no consta,21-30 anos,no consta,no consta,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,no consta,31-40 anos,espana,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Renombramos las siguientes columnas:
vic.rename(columns={'ano': 'año'}, inplace=True)
vic.rename(columns={'tramo de edad agresor': 'edad_agresor'}, inplace=True)
vic.rename(columns={'tramo de edad victima': 'edad_victima'}, inplace=True)
vic.rename(columns={'pareja - expareja': 'pareja'}, inplace=True)
vic.rename(columns={'ano': 'año'}, inplace=True)
vic.rename(columns={'pais de nacimiento agresor': 'nacimiento_agresor'}, inplace=True)
vic.rename(columns={'pais de nacimiento victima': 'nacimiento_victima'}, inplace=True)
vic.rename(columns={'denuncia agresor': 'denuncia'}, inplace=True)

In [38]:
# Transformamos valores en las filas:
vic['edad_agresor'] = eliminar_alfa(vic['edad_agresor'])
vic['edad_victima'] = eliminar_alfa(vic['edad_victima'])
vic.año = clean_year(vic.año)
vic.suicidio = vic.suicidio.map(suicidio)


In [39]:
vic.pareja.unique()

array(['expareja', 'pareja', 'pareja en fase de separacion'], dtype=object)

In [40]:
pareja = {'expareja': 'no',
          'pareja': 'si',
          'pareja en fase de separacion': 'en_separacion'}

vic.pareja = vic.pareja.map(pareja)

In [41]:
vic.denuncia.unique()

array(['no consta denuncia', 'no habia denuncia', 'habia denuncia'],
      dtype=object)

In [42]:
denuncia = {'no consta denuncia': 'desconocido',
          'no habia denuncia': 'no',
          'habia denuncia': 'no'}

vic.denuncia = vic.denuncia.map(denuncia)

In [43]:
vic['edad_victima'].unique()

array(['31-40', '41-50', '21-30', '71-84', '', '61-70', '51-60', '18-20',
       '>=85', '<16', '16-17'], dtype=object)

In [44]:
vic['edad_victima'] = vic['edad_victima'].replace('', 'desconocido')
vic['edad_agresor'] = vic['edad_agresor'].replace('', 'desconocido')

In [45]:
vic['nacimiento_victima'].unique()

array(['espana', 'no consta', 'otro pais'], dtype=object)

In [46]:
nacim = {'espana': 'españa',
         'no consta': 'desconocido',
         'otro pais': 'otro'}


vic['nacimiento_victima'] = vic['nacimiento_victima'].map(nacim)
vic['nacimiento_agresor'] = vic['nacimiento_agresor'].map(nacim)

In [47]:
# Añadimos columna de trimestre:
vic['trimestre'] = vic['mes'].apply(trimestre)

# Vamos a reordenar las columnas:
columnas = vic.columns.tolist()

# Extraemos la última columna de trimestre:
ultima_columna = columnas[-1]
ultima_columna
# La movemos a la tercera posición para que tenga consistencia:
columnas.insert(2, columnas.pop(-1))

# Reordenamos:
vic = vic[columnas]

In [48]:
vic.head()

,año,mes,trimestre,pareja,convivencia,denuncia,suicidio,edad_agresor,edad_victima,nacimiento_agresor,nacimiento_victima,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla
0,2003,enero,primero,no,no,desconocido,no,desconocido,31-40,españa,españa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2003,enero,primero,si,si,desconocido,no,31-40,31-40,otro,españa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2003,enero,primero,si,si,desconocido,no,51-60,41-50,españa,españa,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2003,enero,primero,si,si,desconocido,no,desconocido,21-30,desconocido,desconocido,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2003,enero,primero,si,si,desconocido,no,desconocido,31-40,españa,españa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# Guardamos el nuevo dataframe en un archivo csv

vic.to_csv('../data_clean/portal_estadistico_vio_gen/victimas_mortales.csv', index=False)

### 7. LIMPIEZA ARCHIVO CSV SENTENCIAS

In [50]:
sent = pd.read_csv('../data/sentencias.csv', delimiter='\t')
sent

,Tipo de indicador,Tipo de dato,Periodo,Total
0,Condenados,Dato base,2022,36.161
1,Condenados,Dato base,2021,33.068
2,Condenados,Dato base,2020,25.436
3,Condenados,Dato base,2019,30.495
4,Condenados,Dato base,2018,27.972
5,Condenados,Dato base,2017,27.202
6,Condenados,Dato base,2016,25.959
7,Condenados,Dato base,2015,24.265
8,Absueltos,Dato base,2022,NaN
9,Absueltos,Dato base,2021,4.263


In [51]:
sent = sent.fillna('desconocido')

In [52]:
sent = sent.apply(lower_tildes)
sent.columns = unif_col(sent.columns)

In [53]:
sent

,tipo de indicador,tipo de dato,periodo,total
0,condenados,dato base,2022,36.161
1,condenados,dato base,2021,33.068
2,condenados,dato base,2020,25.436
3,condenados,dato base,2019,30.495
4,condenados,dato base,2018,27.972
5,condenados,dato base,2017,27.202
6,condenados,dato base,2016,25.959
7,condenados,dato base,2015,24.265
8,absueltos,dato base,2022,desconocido
9,absueltos,dato base,2021,4.263


In [54]:
# Renombramos las siguientes columnas:
sent.rename(columns={'tipo de indicador': 'indicador'}, inplace=True)
sent.rename(columns={'periodo': 'año'}, inplace=True)




In [55]:
sent = sent[['año', 'indicador', 'total']]
sent = sent.sort_values(by = 'año').reset_index(drop=True)
sent

,año,indicador,total
0,2015,condenados,24.265
1,2015,absueltos,6.768
2,2016,condenados,25.959
3,2016,absueltos,6.288
4,2017,condenados,27.202
5,2017,absueltos,6.181
6,2018,condenados,27.972
7,2018,absueltos,5.025
8,2019,condenados,30.495
9,2019,absueltos,5.149


In [56]:
# Guardamos el nuevo dataframe en un archivo csv

sent.to_csv('../data_clean/ine/sentencias.csv', index=False)

### 8. LIMPIEZA ARCHIVO CSV TIPOS DE VIOLENCIAS

In [58]:
tipos = pd.read_csv('../data/tipos.csv', encoding= 'utf-8', sep=';')
tipos

,Tipo de infracción,Comunidades y ciudades autónomas,Periodo,Total
0,Delitos,Andalucía,2022,9.955
1,Delitos,Andalucía,2021,9.260
2,Delitos,Andalucía,2020,7.103
3,Delitos,Andalucía,2019,8.539
4,Delitos,Andalucía,2018,7.856
...,...,...,...,...
2731,Otras faltas sin especificar,Melilla,2019,0.000
2732,Otras faltas sin especificar,Melilla,2018,0.000
2733,Otras faltas sin especificar,Melilla,2017,0.000
2734,Otras faltas sin especificar,Melilla,2016,0.000
